In [1]:
### Required Libraries ###
### This is how the requests library is imported to AWS Lambda ###
from datetime import datetime
from dateutil.relativedelta import relativedelta
from botocore.vendored import requests

ModuleNotFoundError: No module named 'botocore'

In [2]:
### This section is for helper functions ###

### This section is for data validation ###

- Values should be greater than 0
-  Age should be between 18 and 100 

#I dont understand this one. Relates to the output of the validate data. 

def build_validation_result(is_valid, violated_slot, message_content):
    """
    Defines an internal validation message structured as a python dictionary.
    """
    if message_content is None:
        return {"isValid": is_valid, "violatedSlot": violated_slot}

    return {
        "isValid": is_valid,
        "violatedSlot": violated_slot,
        "message": {"contentType": "PlainText", "content": message_content},
    }
## Validates the data provided by the customer. 
    
def validate_data(debtTotal,humanAge,incomeMonthly,debtAvailable, debtmonthly):
    
    if debtTotal is not None:
        debtTotal = parse_float(
            debtTotal
        )  # Since parameters are strings it's important to cast values
        if debtTotal < 0:
            return build_validation_result(
                False,
                "debtTotal"
                "The total amount of Debt amount cannot be negative. "
                "Please provide a number greater than or equal to 0.",
            )
        
    if humanAge is not None:
        humanAge = parse_float(
            humanAge
        )  # Since parameters are strings it's important to cast values
        if humanAge < 0:
            return build_validation_result(
                False,
                "humanAge",
                "This service requires you to be 18 or older. Please enter a valid age.",
            )
        
    if incomeMonthly is not None:
        incomeMonthly = parse_float(
            incomeMonthly
        )  # Since parameters are strings it's important to cast values
        if incomeMonthly < 0:
            return build_validation_result(
                False,
                "incomeMonthly",
                "Monthly income cannot be negative. "
                "Please enter a number greater than or equal to 0.",
            )
        
    if debtAvailable is not None:
        debtAvailable = parse_float(
            debtAvailable
        )  # Since parameters are strings it's important to cast values
        if debtAvailable < 0:
            return build_validation_result(
                False,
                "debtAvailable",
                "The debtAvailable amount cannot be negative. "
                "Please enter a number greater than or equal to 0.",
            )
        
    if debtmonthly is not None:
        debtmonthly = parse_float(
            debtmonthly
        )  # Since parameters are strings it's important to cast values
        if debtmonthly < 0:
            return build_validation_result(
                False,
                "debtmonthly",
                "The debtmonthly amount cannot be negative. "
                "Please enter a number greater than or equal to 0.",
            )

    return build_validation_result(True, None, None)


SyntaxError: invalid syntax (<ipython-input-2-d07027c01839>, line 5)

In [3]:
### this section is for helper functions to control the conversation to response events
def get_slots(intent_request):
    """
    Fetch all the slots and their values from the current intent.
    """
    return intent_request["currentIntent"]["slots"]
## not sure what this does
def elicit_slot(session_attributes, intent_name, slots, slot_to_elicit, message):
    """
    Defines an elicit slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "ElicitSlot",
            "intentName": intent_name,
            "slots": slots,
            "slotToElicit": slot_to_elicit,
            "message": message,
        },
    }

## not sure what this does
def delegate(session_attributes, slots):
    """
    Defines a delegate slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {"type": "Delegate", "slots": slots},
    }


def close(session_attributes, fulfillment_state, message):
    """
    Defines a close slot type response.
    """

    response = {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "Close",
            "fulfillmentState": fulfillment_state,
            "message": message,
        },
    }

    return response

def predict_delinquency(intent_request):
    """
    Performs dialog management and fulfillment for predicting financial future.
    """
    # Gets slots' values
    humanAge = get_slots(intent_request)["humanAge"]
    debtMonthly = get_slots(intent_request)["debtMonthly"]
    incomeMonthly = get_slots(intent_request)["incomeMonthly"]
    debtAvailable = get_slots(intent_request)["debtAvailable"]
    debtTotal = get_slots(intent_request)["debtTotal"]

    # Gets the invocation source, for Lex dialogs "DialogCodeHook" is expected.
    source = intent_request["invocationSource"]

    if source == "DialogCodeHook":
        # This code performs basic validation on the supplied input slots.

        # Gets all the slots
        slots = get_slots(intent_request)

        # Validates user's input using the validate_data function
        validation_result = validate_data(humanAge, debtMonthly, incomeMonthly,debtAvailable,debtTotal)

        # If the data provided by the user is not valid,
        # the elicitSlot dialog action is used to re-prompt for the first violation detected.
        if not validation_result["isValid"]:
            slots[validation_result["violatedSlot"]] = None  # Cleans invalid slot

            # Returns an elicitSlot dialog to request new data for the invalid slot
            return elicit_slot(
                intent_request["sessionAttributes"],
                intent_request["currentIntent"]["name"],
                slots,
                validation_result["violatedSlot"],
                validation_result["message"],
            )

        # Fetch current session attributes
        output_session_attributes = intent_request["sessionAttributes"]

        # Once all slots are valid, a delegate dialog is returned to Lex to choose the next course of action.
        return delegate(output_session_attributes, get_slots(intent_request))
    
    
    
    ####
    #### TEAM, THIS IS, I THINK, WHERE WE PERFORM THE ML PORTION. MAYBE OUTSIDE THE FUNCTION. I DUNNO. 
    #####

    # Return a message with the Genie's result. Need to identify where Fulfilled is coming from/going to
    return close(
        intent_request["sessionAttributes"],
        "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """Thanks for waiting;
            Our models are predicting a %{} chance of you experiencing significant financial turmoil in the next 2 years. Have a good day!.
            """.format(
                delinquency_prediction
            ),
        },
    )


In [ ]:


### Intents Dispatcher ###
def dispatch(intent_request):
    """
    Called when the user specifies an intent for this bot.
    """

    # Get the name of the current intent
    intent_name = intent_request["currentIntent"]["name"]

    # Dispatch to bot's intent handlers
    if intent_name == " DeliquencyGenie":
        return predict_delinquency(intent_request)

    raise Exception("Intent with name " + intent_name + " not supported")
    
### Main Handler ###
def lambda_handler(event, context):
    """
    Route the incoming request based on intent.
    The JSON body of the request is provided in the event slot.
    """

    return dispatch(event)